In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import os
import scipy as sp
from tqdm import tqdm

# 1. Investigating Diabetes Trends in the Synthea Data

In this module, we will:
1. Load and explore synthetic EHR data generated using Synthea.
2. Identify patients diagnosed with Type 2 Diabetes using SNOMED CT codes.
3. Analyze Hemoglobin A1C (HbA1c) levels in diabetic and non-diabetic patients.
4. Visualize data distributions and perform statistical testing.
5. Discuss findings and explore possible extensions.

## Step 1: Loading of Relevant Data
Data for 4,000 synthetic patients, divided into four states, was generated using the [Synthea tool](https://synthea.mitre.org/)
This data is split into four separate directories
```
output_hi/ # patients in Hawaii
output_ma/ # patients in Massacusetts
output_tx/ # patients in Texas
output_wa/ # patients in Washington
```

In the introductory primer, we worked exclusively with the patient demographics data stored in patients.csv
For the subsequent investigation, we will be relying on two new files
- `conditions.csv`: Stores SNOMED CT coded patient conditions and diagnoses
- `observations.csv`: Includes vital signs and LOINC coded lab results for all patients \
Below is a convenience function that can be used to load the concatenated conditions and observations for all states
into a single data frame \
This uses a new operation that we haven't seen previously, `pd.concat()` which is used to concatenate DataFrames \
together, row-wise

In [ ]:
def load_data_for_file(filename):
    print(f"Loading data for {filename}")
    df = pd.concat([ # use pd.concat to append/concatenate the data for all states together into a single frame
        pd.read_parquet(f"https://dicbworkshops.s3.amazonaws.com/{output_dir}/parquet/{filename}") # use read_csv to load the data from each output directory
        for output_dir in tqdm(['output_hi', 'output_ma', 'output_tx', 'output_wa']) # loop over each output directory
    ])
    return df

In [ ]:
# load in the conditions and observations
conditions = load_data_for_file('conditions.parquet')
observations = load_data_for_file('observations.parquet')

In [ ]:
conditions

In [ ]:
observations

For this investigation, we are interested in patients who have been diagnosed with Type 2 Diabetes \
In order to do this, we filter the conditions table to get all diagnoses of type 2 diabetes \
We can use the SNOMED CT code for type 2 diabetes: `44054006`

In [ ]:
diabetes = conditions.query('CODE == 44054006') # we could also use a substring search if we didn't know the code

In [ ]:
diabetes

In [ ]:
# count the unique patients with Type 2 diabetes 
# (NOTE: Not always the same as the number shown above for rows, as patients will often have repeat diagnoses depending on the condition/disorder)
diabetes['PATIENT'].nunique()

Now we will analyze Hemoglobin A1C levels for patients with and without a diabetes diagnosis \
To do this, we first filter the observations table for all hemoglobin A1C lab results \
We can use LOINC code: `4548-4`

In [ ]:
hemoglobin_a1c = observations[observations['CODE'] == '4548-4']

In [ ]:
hemoglobin_a1c

Next we will add another variable/column to the hemoglobin_a1c data to label patients with/without a Type 2 diabetes diagnosis

In [ ]:
hemoglobin_a1c_labeled = hemoglobin_a1c.assign(
    HASDIABETES=hemoglobin_a1c['PATIENT'].isin(diabetes['PATIENT']).astype('int')
)

In [ ]:
hemoglobin_a1c_labeled

Now we will plot the distributions of A1C in the two groups using the Seaborn package \
and display summary statistics for both

In [ ]:
sns.kdeplot(hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'), label="With Diabetes", color="red", alpha=0.4, common_norm=False)
sns.kdeplot(hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float'), label="Without Diabetes", color="blue", alpha=0.4, common_norm=False)
plt.legend()
plt.show()

In [ ]:
sns.violinplot(
    hemoglobin_a1c_labeled.astype({'VALUE': float}),
    x='HASDIABETES',
    y='VALUE'
)

plt.show()

In [ ]:
hemoglobin_a1c_labeled.astype({'VALUE': 'float'}).groupby('HASDIABETES')['VALUE'].describe()

If we want to quantify whether there is a significant difference in the two distributions \
one option is the use the Mann-Whitney U (Wilcoxon rank-sum) test \
provided by scipy's mannwhitneyu() function

In [ ]:
from scipy.stats import mannwhitneyu

In [ ]:
U1, p = mannwhitneyu(
    hemoglobin_a1c_labeled.query('HASDIABETES == 1')['VALUE'].astype('float'),
    hemoglobin_a1c_labeled.query('HASDIABETES == 0')['VALUE'].astype('float'), 
    method="auto"
)

In [ ]:
print(f"U1: {U1}, p-value: {p}")

# Discussion Points
1. Why is it that we observe a bi-modal distribution in A1C values among patients without a Type 2 Diabetes diagnosis?
2. Why do we observe right skewness and high A1C outliers in diabetic patients, while the medians of the two distributions are identical? What biological or clinical factors might explain this?
3. How can we extend this analysis to add additional nuance (additional lab tests, adding data from other tables, partitioning patients further based on \
   other diagnoses, medications, demographics etc?

# 2. Extensions to Initial Analysis
Try to see if you can extend the initial analysis that we did on Hemoglobin A1C and diabetes

1. Are there any other labs that you could pull data for and compare distributions for between Type-2 and non-Type2 patients
2. Could you further subdivide the Type 2 and non-Type 2 patients into additional subsets based on other diagnoses/demographics/medication status?


**Click Below for Hints**

<details>
    <summary>Hint 1</summary>
    <p>If you want to load an additional file, say the medications table, you can use <code>medications = load_data_for_file('medications.parquet')</code></p>
</details>
<details>
    <summary>Hint 2</summary>
    <p>The medications table has both a <code>'REASONCODE'</code> abd <code>'REASONDESCRIPTION'</code> column which describe why the medication was prescribed</p>
</details>